# Librerias

In [16]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import signal
import os
%matplotlib widget

# Declaracion de constantes 

In [17]:
F_MIN = 5
F_MAX = 40
F_MUESTREO = 128
RUTA_DATOS_PROCESADOS = '../DatosProcesados/Frecuencia'
CARPETAS = ['Rojo', 'Azul', 'Verde', 'Morado', 'Baseline']

# Declaracion de Funciones 

In [50]:
def plot_datos(datos):
    
    plt.figure()
    plt.plot(datos[:, 0], label = 'EEG.O1')
    plt.plot(datos[:, 1], label = 'EEG.O2')
    plt.legend()
    plt.title('Raw Data')
    plt.xlabel('Tiempo')
    plt.ylabel('uV')
    
def plot_psd_semilog(datos):
    
    f1, psd_O1 = signal.periodogram(datos[:, 0], F_MUESTREO)
    f2, psd_O2 = signal.periodogram(datos[:, 1], F_MUESTREO)
    plt.semilogy(f1, psd_O1, label = 'EEG.O1')
    plt.semilogy(f1, psd_O2, label = 'EEG.O2')
    plt.ylim([1e-7, 1e2])
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.legend()

    
def plot_psd(datos):
    
    plt.figure()
    f1, psd_O1 = signal.periodogram(datos[:, 0], F_MUESTREO)
    f2, psd_O2 = signal.periodogram(datos[:, 1], F_MUESTREO)
    plt.plot(f1, psd_O1, label = 'EEG.O1')
    plt.plot(f1, psd_O2, label = 'EEG.O2')
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD')
    plt.legend()
    print(psd_O1.mean())
    print(psd_O2.mean())
    print(max(psd_O1))
    print(max(psd_O2))

def filtro_pasa_banda(datos, f_min, f_max, f_muestreo, orden=5):
    nyq = 0.5 * f_muestreo
    f_min = f_min / nyq
    f_max = f_max / nyq
    b, a = signal.butter(orden, [f_min, f_max], btype='band')
    print(b, a)
    y = signal.lfilter(b, a, datos)
    return y

def hilbert_from_scratch(u):
    # N : fft length
    # M : number of elements to zero out
    # U : DFT of u
    # v : IDFT of H(U)

    N = len(u)
    # take forward Fourier transform
    U = fftpack.fft(u)
    M = N - N//2 - 1
    # zero out negative frequency components
    U[N//2+1:] = [0] * M
    # double fft energy except @ DC0
    U[1:N//2] = 2 * U[1:N//2]
    # take inverse Fourier transform
    v = fftpack.ifft(U)
    return v

## Graficas de Datos (Sin filtro)

In [60]:
from scipy import fftpack 

for carpeta in CARPETAS:
    archivos = [e for e in os.listdir(RUTA_DATOS_PROCESADOS+'/'+carpeta) if e.endswith('.csv') ]
    
    for indice, archivo in enumerate(archivos):
        ruta_origen = f'{RUTA_DATOS_PROCESADOS}/{carpeta}/{archivo}'
        print(archivo)
        df = pd.read_csv(ruta_origen)
        etiqueta = df[['MarkerValueInt']]
        datos = df[['EEG.O1', 'EEG.O2']].to_numpy()
        o1 = fftpack.hilbert(datos[:, 0])
        o1 = pd.DataFrame(data=o1, columns=["EEG.O1"])
        o2 = fftpack.hilbert(datos[:, 0])
        o2 = pd.DataFrame(data=o2, columns=["EEG.O2"])
        archivo = pd.concat([o1, o2, etiqueta], axis=1).to_csv(f'{RUTA_DATOS_PROCESADOS}/{carpeta}/hilbert{indice}.csv', index = False)
        break
    break

muestra11.csv


In [5]:
!rm ../DatosProcesados/Frecuencia/Rojo/sinfiltro/raw/*.png

In [ ]:
df = pd.read_csv('../DatosProcesados/Frecuencia/Filtered_Azul.csv')
df2 = pd.read_csv('../DatosProcesados/Frecuencia/ejemplo6(azul).csv')
datos = df.to_numpy()
datos2 = df2[['EEG.O1', 'EEG.O2']].to_numpy()
y = signal.hilbert(datos)
plt.plot(y)
plot_datos(datos)
plot_psd(datos2)
plot_psd(datos)
plot_psd(y)


In [23]:
datos[0][0]

4172.436035

In [25]:
y[0][0]

(4172.436034999999+0j)

In [29]:
archivo

,EEG.O1,EEG.O2,MarkerValueInt
0,4172.436035+0.000000j,4184.230957+0.000000j,1.0
1,4169.615234+0.000000j,4181.410156+0.000000j,1.0
2,4168.461426+0.000000j,4183.974121+0.000000j,1.0
3,4172.563965+0.000000j,4183.974121+0.000000j,1.0
4,4176.666504+0.000000j,4189.871582+0.000000j,1.0
...,...,...,...
440,4168.461426+0.000000j,4169.487305+0.000000j,1.0
441,4169.743652+0.000000j,4171.153809+0.000000j,1.0
442,4169.358887+0.000000j,4174.487305+0.000000j,1.0
443,4164.615234+0.000000j,4171.410156+0.000000j,1.0


In [44]:
import math
N = 32
f = 1
dt = 1.0 / N
y = []
for n in range(N):
    x = 2*math.pi*f*dt*n
    y.append(math.sin(x))
z1 = hilbert_from_scratch(y)
z2 = hilbert(y)

In [52]:
y


array([4172.43505622+4.52226739e+00j, 4169.61623652+8.91796229e+00j,
       4168.46039979+4.87170474e+00j, 4172.56501484+3.36993008e+00j,
       4176.66543057+6.49996783e+00j, 4174.61633096+1.16844277e+01j,
       4170.38364557+9.32129661e+00j, 4174.74479586+5.22656997e+00j,
       4180.12725078+1.11679529e+01j, 4174.87277252+1.58097769e+01j,
       4174.87036823+1.18057367e+01j, 4180.12965495+1.65814492e+01j,
       4174.87032192+2.46831786e+01j, 4166.79620514+2.15772113e+01j,
       4170.51138886+1.77220546e+01j, 4170.25767707+2.56530709e+01j,
       4159.87023007+2.52314749e+01j, 4158.46280073+1.84381829e+01j,
       4157.81891554+1.75878325e+01j, 4155.00142012+1.12875983e+01j,
       4161.92140929+4.78837272e+00j, 4169.10400423+8.09402355e+00j,
       4170.12693033+1.19226153e+01j, 4172.43754504+1.36679237e+01j,
       4175.12688566+2.06466541e+01j, 4169.74520656+2.84745596e+01j,
       4163.0755713 +2.99474500e+01j, 4157.43763376+3.13299308e+01j,
       4149.35726626+2.98918895e+0

In [55]:
y.tolist()

[-4.525633289679341,
 -8.914603383462188,
 -4.8750564779111,
 -3.366585664392774,
 -6.503304747186103,
 -11.68109842757955,
 -9.324618041113624,
 -5.223256536118303,
 -11.171258207648988,
 -15.806479927124771,
 -11.809025149865429,
 -16.57816936837525,
 -24.686449554595253,
 -21.573949238749705,
 -17.725307535320884,
 -25.649827354911032,
 -25.23470898699884,
 -18.434958359528046,
 -17.591047228112625,
 -11.284393497810536,
 -4.791567383392453,
 -8.09083915923668,
 -11.925789293178674,
 -13.664760310899773,
 -20.649806727910047,
 -28.47141787136424,
 -29.95058068064239,
 -31.326811277930187,
 -29.894997591102864,
 -23.928423102496787,
 -15.41872182843023,
 -8.106461547777975,
 -5.143746889167721,
 -6.705183308635154,
 -9.84687224051637,
 -9.610177906649765,
 -6.196449438139843,
 -6.51952254489509,
 -8.59790361959945,
 -6.707324672753115,
 -2.497141311551056,
 -4.086545967927383,
 -7.0689668414940785,
 -5.228947173887303,
 -7.858018948419396,
 -10.618001647207292,
 -3.1992811337444746,


In [58]:
o1.tolist()

[-4.525633289679341,
 -8.914603383462188,
 -4.8750564779111,
 -3.366585664392774,
 -6.503304747186103,
 -11.68109842757955,
 -9.324618041113624,
 -5.223256536118303,
 -11.171258207648988,
 -15.806479927124771,
 -11.809025149865429,
 -16.57816936837525,
 -24.686449554595253,
 -21.573949238749705,
 -17.725307535320884,
 -25.649827354911032,
 -25.23470898699884,
 -18.434958359528046,
 -17.591047228112625,
 -11.284393497810536,
 -4.791567383392453,
 -8.09083915923668,
 -11.925789293178674,
 -13.664760310899773,
 -20.649806727910047,
 -28.47141787136424,
 -29.95058068064239,
 -31.326811277930187,
 -29.894997591102864,
 -23.928423102496787,
 -15.41872182843023,
 -8.106461547777975,
 -5.143746889167721,
 -6.705183308635154,
 -9.84687224051637,
 -9.610177906649765,
 -6.196449438139843,
 -6.51952254489509,
 -8.59790361959945,
 -6.707324672753115,
 -2.497141311551056,
 -4.086545967927383,
 -7.0689668414940785,
 -5.228947173887303,
 -7.858018948419396,
 -10.618001647207292,
 -3.1992811337444746,
